# Agenda, day 2

1. Recap
2. Q&A
3. Magic methods
4. Class attributes
5. Finding attributes with ICPO
6. Inheritance -- and how ICPO influences it
7. Three models for method inheritance
8. Data inheritance
9. AMA / where to from here?

# Recap

How do we solve problems? Many times, in the computer world, it's easiest to solve problems if we have data structures that are aligned with our thinking. By creating custom data structures, on top of the ones that Python provides, we can (a) write code more easily, (b) understand/maintain code more easily, and even (c) get better performance out of our code.

By doing this, we make our code easier to write and maintain. 

A few points to think about and to recap:

- By creating a new data structure, we gain the power of abstraction, of ignoring the lower-level thinking and concentrating on the higher-level thinking.
- Methods (verbs) are defined on the class, which means they are tightly bound to one particular data type. Regular functions can be invoked with any argument, until/unless something goes wrong -- sometimes because you invoked it with the wrong kind of argument! But in the case of methods, when you invoke a method, the instance on which you ran it is automatically given to the `self` parameter as a value. Other arguments are also OK, but the main one is automatically sent.
- We can define new data types, aka "classes," which are also known as "types." Each type defines a combination of data structure (on top of existing ones) and methods (sometimes new, sometimes copied from elsewhere). You can use `type` as as function to find out what kind of data structure something is. You can only use `class` when defining a new class in Python.
- The most important method on a class is `__init__` ("dunder init"). This method is invoked by Python after a new object has been created. Its job is to add attributes to the new instance that we have created. `__init__` will always have `self` as its first parameter, which in its case will be not only the instance, but the *new* instance without any attributes at all. In `__init__`, we want to assign any/all attributes that might be used on the instance down the road. `__init__` doesn't need to return anything; its job is only to assign attributes.
- Each time we invoke a method, the first argument is populated with the instance on which we're running things. `self` is the conventional name for this parameter, and it's a bad idea to give it another name. But Python won't check or test things; if you forget to put `self` as the first parameter, then whatever the first parameter is, that'll get the instance assigned to it.
- Most attributes are set based on the parameters in `__init__`. But there's no rule that says all parameters are to be used for attributes, or that all attributes need to come from parameters.

In [1]:
class Person:
    def __init__(self, name, shoe_size):  # these parameters are assigned values by the caller via arguments
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0           # want to assign to an attribute on the instance? Use self!

    def greet(self):
        return f'Hello, {self.name}!'   # want to retrieve from an attribute on the instance? Use self!

In [2]:
# parameters:   name    shoe_size
# arguments:   'Reuven'  46

p = Person('Reuven', 46)

In [3]:
p.greet()     # --> Person.greet(p)

'Hello, Reuven!'

In [4]:
p.name = 'whoever'

In [5]:
p.greet()

'Hello, whoever!'

In [6]:
vars(p)

{'name': 'whoever', 'shoe_size': 46, 'bank_balance': 0}

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on your instance of `Cellphone`. It will return a string saying, `calling` and the number that you provide as an argument.
3. Define an `info` method that returns a dict containing the phone's number and model -- this will be useful if it's stolen.

In [9]:
class Cellphone:
    def __init__(self):
        self.model = '(Undefined)'
        self.number = '(No number)'

c = Cellphone()
c.model = 'FancyInc phone'
c.number = '12345'

vars(c)

{'model': 'FancyInc phone', 'number': '12345'}

In [10]:
# better (I think) normally to get as many attribute values as possible via parameters

class Cellphone:
    def __init__(self, model, number):
        self.model = model
        self.number = number

c = Cellphone('FancyInc phone', '12345')
vars(c)

{'model': 'FancyInc phone', 'number': '12345'}

In [11]:
class Cellphone:
    def __init__(self, model, number):
        self.model = model
        self.number = number

c = Cellphone('FancyInc phone', '12345')
c.call('2468')

AttributeError: 'Cellphone' object has no attribute 'call'

In [13]:
class Cellphone:
    def __init__(self, model, number):
        self.model = model
        self.number = number
        
    def call(self, other_number):
        print(f'{self.number} calling {other_number}...')

    def info(self):
        return vars(self)   # returns a dict with our attributes!

c = Cellphone('FancyInc phone', '12345')
c.call('2468')
c.info()

12345 calling 2468...


{'model': 'FancyInc phone', 'number': '12345'}

In [15]:
class Cellphone:
    def __init__(self, model, number):
        self.model = model
        self.number = number
        
    def call(self, other_number):
        print(f'{self.number} calling {other_number}...')

    def info(self):
        return vars(self)   # returns a dict with our attributes!

c1 = Cellphone('fancy 1000', '12345')
c2 = Cellphone('old 123', '123')

c1.call(c2.number)

12345 calling 123...


# Magic methods

You might have noticed that strings, lists, tuples, and dicts all support `[]` to retrieve and set values. How can that be? The answer is that in every case of a Python operator -- the different symbols we use -- the operation is turned into a method call. This means that if an appropriately named method is defined, then that method will be called.

These methods are almost never called directly by us. Rather, they are called by Python when it encounters certain circumstances or operators. 

These methods are known as "dunder methods" or "magic methods." The idea is that once you define a magic method, Python will invoke it on your behalf at various points in your program.

We've already encountered `__init__`, which is invoked automatically after a new object is created, but before it's returned to the caller. We will almost never invoke `__init__` on our own. Rather, we'll let Python do it for us.

Someone once described magic methods to me as "callback functions," meaning that we define them and the system decides when to invoke them.

We're going to look at a few magic methods, and how we can use them to make objects that are more interesting and useful. Also, by defining magic methods, we can write objects that integrate into Python's conventions more readily.

# Magic method: `__len__`

When you invoke the `len` function on an object, `len` doesn't really know what to do. Rather, it turns to the object, and sees if it has a `__len__` method. If so, and if `__len__` returns an integer, then it's run. 



In [16]:
s = 'abcd'

len(s)

4

In [17]:
# I can write the code this way, but I should not:

s.__len__()

4

# How to add `__len__` to your object

1. Define a method, `__len__`. It should have one parameter, `self`.
2. It can calculate whatever you want, however you want. (Ideally, it'll return something normal/useful for the object.)
3. It must return an integer. If it doesn't, then Python will give you an error.
4. When you invoke `len` (the function) on your object, `__len__` will then be invoked.



# Exercise: Make `len` work on `Bowl` objects

1. Modify the `Bowl` object from yesterday, such that we can invoke `len` on it.
2. Do so, and see that it works.
3. Would it make sense to implement `__len__` on `Scoop`? If so, what would be a reasonable value?

In [ ]:
# yesterday, we said

len(b.scoops)  # this returned the length of the list, b.scoops

# today, I want to say

len(b)   # this will return the same number, the number of scoops

In [20]:
# From .. 
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)
b.flavors()
len(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...


3

# Next up

- `__str__` and `__repr__` 

# Strings and objects

You might have noticed that *every* object in Python can be passed to `print`, and printed out. Similarly, every object in Python can be passed to `str`, and we get a string version of it. 

- The reason for this is that `print` converts every argument it gets into a string, by invoking `str`.
- When you invoke `str` on an object, Python looks for its `__str__` magic method.



In [21]:
print(5)

5


In [22]:
print([10, 20, 30])

[10, 20, 30]


In [23]:
print({'a':10, 'b':20, 'c':30})

{'a': 10, 'b': 20, 'c': 30}


In [24]:
# when I say print(d), a dict,

# that really means: print(str(d)), which invokes print(d.__str__())

# so, if we want our objects to be printable in a nice way, beyond the default, all we have to do is define __str__

In [26]:
print(c)

In [27]:
vars(c)

{'model': 'FancyInc phone', 'number': '12345'}

In [28]:
class Cellphone:
    def __init__(self, model, number):
        self.model = model
        self.number = number
        
    def call(self, other_number):
        print(f'{self.number} calling {other_number}...')

    def info(self):
        return vars(self)   # returns a dict with our attributes!

    def __str__(self):
        return f'Cellphone, model {self.model}, number {self.number}'

c1 = Cellphone('fancy 1000', '12345')
c2 = Cellphone('old 123', '123')

c1.call(c2.number)

12345 calling 123...


In [29]:
print(c1)

Cellphone, model fancy 1000, number 12345


In [30]:
print(c2)

Cellphone, model old 123, number 123


In [31]:
str(c1)

'Cellphone, model fancy 1000, number 12345'

In [32]:
str(c2)

'Cellphone, model old 123, number 123'

# Exercise: Printable scoops

1. Modify the `Scoop` class, such that printing a `Scoop` instance, or just invoking `str` on a `Scoop` instance, returns and/or prints something like `Scoop of FLAVOR`.
2. Test it on your scoops.

In [33]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

print(s1)
print(s2)
print(s3)

In [34]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

print(s1)
print(s2)
print(s3)

Scoop of cookie dough
Scoop of chocolate
Scoop of vanilla


# Exercise: Printable bowls

1. Define `__str__` on the `Bowl` class, such that saying `print(b)` will print:
    - The fact that it's a bowl
    - Each of the scoops in the bowl on a line by itself
    - If you can number the scoops, even better!
2. Demo that it works

Example:

    print(b)

    Bowl with:
    1. Scoop of chocolate
    2. Scoop of vanilla
    3. Scoop of coffee


In [39]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __str__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [40]:
print(s1)

Scoop of cookie dough


In [41]:
print(b)

Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [42]:
s1

In [43]:
b

# `__str__` vs. `__repr__`

There are two different ways that Python will try to turn our objects into strings:

- The main way, and the way that most people think/know about, is via `__str__`. We've seen how this works. The output from `__str__` is supposed to be good enough to show end users.
- If you're in a Python debugger, or in Jupyter, and you don't use `print` or `str`, but you just ask to see the printed representation of an object, then you're going to see the `__repr__`. This is handled by a different method. The output from `__repr__` isn't supposed to be beautiful, but rather meaningful for programmers.

Consider:

- If you define both `__str__` and `__repr__`, then each is invoked at different times.
- If you define only `__str__`, then it'll work when you `print` or `str`, but not when you are looking at values in Jupyter or the debugger.
- If you only define `__repr__`, then it handles **ALL** cases!

And so, my suggestion (which goes against the official Python recommendations) is to just define `__repr__`. Most of the time, you don't need to distinguish between what is shown to users and what is shown to coders. And if/when you ever do need to make that distinction, then just rename `__repr__` to `__str__`, and do something special/new with `__repr__` instead.

# Exercise: Use `__repr__`!

- Before doing anything else, see that there is a difference between `print`ing your object and just getting the printed representation. (If you're not in Jupyter, then you can invoke `repr(b)` to get its representation.
- Rename `__str__` to be `__repr__`. Now do you see the difference in output?

In [44]:
print(s1)

Scoop of cookie dough


In [45]:
s1

In [46]:
print(b)

Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [47]:
b

In [48]:
b.scoops

In [49]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [50]:
s1

Scoop of cookie dough

In [51]:
s2

Scoop of chocolate

In [52]:
s3

Scoop of vanilla

In [53]:
b

Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla

In [54]:
b.scoops

[Scoop of cookie dough, Scoop of chocolate, Scoop of vanilla]

# Other magic methods

Every single operator you use in Python is translated into a magic method.

For example, I mentioned earlier that many different classes use `[]` to retrieve something (or even set something). How can that be? They all implement the magic method `__getitem__`. 

In [55]:
s = 'abcde'

d = {'a':10, 'b':20, 'c':30}

In [56]:
s[0]

'a'

In [57]:
d['a']

10

In [58]:
s['a']

TypeError: string indices must be integers, not 'str'

In [59]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

    def __getitem__(self, index):
        return self.scoops[index]

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [60]:
b[1]

Scoop of chocolate

In [61]:
b[1:]

[Scoop of chocolate, Scoop of vanilla]

In [62]:
b1 = Bowl()
b2 = Bowl()

b1.add_scoops(s1, s2, s3)
b2.add_scoops(s1, s2, s3)

In [63]:
# should we consider these bowls to be equal?

b1 == b2    

False

Behind the scenes, when you invoke `==`, Python looks for the magic method `__eq__`. It is invoked on the value on the *left* side.

In [64]:
b1.__eq__(b2)   # don't do this, but this is what's happening behind the scenes!

NotImplemented

In [65]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

    def __getitem__(self, index):
        return self.scoops[index]

    def __eq__(self, other):
        return self.flavors() == other.flavors()

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [66]:
b1 = Bowl()
b2 = Bowl()

b1.add_scoops(s1, s2, s3)
b2.add_scoops(s1, s2, s3)

In [67]:
b1 == b2

True

In [68]:
b1 = Bowl()
b2 = Bowl()

b1.add_scoops(s1, s2, s3)
b2.add_scoops(s2, s3, s1)

b1 == b2

False

In [69]:
# what will happen in this case?

b1 == 'hello'   # we're running b1.flavors() == 'hello'.flavors() behind the scenes

AttributeError: 'str' object has no attribute 'flavors'

In [70]:
# usually we want to be forgiving when someone uses == or a similar operator
# with values of different types

class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.scoops.append(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

    def __getitem__(self, index):
        return self.scoops[index]

    def __eq__(self, other):
        if isinstance(other, Bowl):    # if these are both instances of Bowl... then check flavors
            return self.flavors() == other.flavors()

        return False

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [72]:
b1 = Bowl()
b2 = Bowl()

b1.add_scoops(s1, s2, s3)
b2.add_scoops(s1, s2, s3)

b1 == b2

True

In [73]:
b1 == 'hello'

False

# Next up

- Class attributes
- Attribute lookup (ICPO)
- Inheritance
- Final questions/etc.

# What are `__main__` and `__file__`?

1. `__file__` is a variable, available everywhere in Python, whenever your program is running. It tells you what file is currently running. It's mostly used when you write modules (i.e., separate files of Python code that you can load via the `import` statement).

2. `'__main__'` is *not* a variable, or a method! It's a string! Typically, you check whether the variable `__name__` (also not a method) is defined to be the current filename (in which case it was imported as a module) or was the first file to run (in which case its value is the string `'__main__'`.



# Object-oriented programming and modules

Modules in Python allow us to split up tasks and definitions across different files, and thus avoid reinventing the wheel. You can `import` a module that you wrote or one that comes with Python ("standard library") or that you downloaded/installed from PyPI on the Internet.

A module can contain function definitions, variable definitions, or even class definitions.   But they are not classes!

A class is typically defined inside of a module, but you can do it in Jupyter, as I've been doing here. You don't need a separate file for it.



# My talk from PyCon IL 3-4 years

Look for this talk on YouTube -- on my YT channel, I have a playlist of my conference talks. 

"What happens when you `import` a module?" will answer a lot of questions about modules.

# Class attributes

We've seen a few things:

- Everything in Python is an object! Which means that everything follows the same rules.
- When we create an instance of an object, the behavior is determined by `__init__` on the class.
- In `__init__`, we assign one or more attributes to `self`, the instance.
- We could, in theory, assign those attributes later on, but it's good practice to do so when the class is defined.

I'll now tell a story that will go into how attributes work not just on instances, but on classes, as well.

In [74]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


Our customers love our `Person` class! BUT.. they want to know how many people have been created in our virtual universe. We'd like to know the population of people at any given time.

In [76]:
# how can I do this?
# option 1: global variable, "population"

population = 0

class Person:
    def __init__(self, name):
        population += 1    # let's add 1 to population
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0


UnboundLocalError: cannot access local variable 'population' where it is not associated with a value

# What's going on?

If you assign to a variable inside of a function, that variable is immediately tagged as a *local* variable. This is picked up by Python's compiler when you define the function. Any assignment to a variable, anywhere in the function, gives it that status.

In our function, we used `+=` on `population`. Because we used assignment on `population`, it was seen as local. This means that when we said `+=`, we wanted to retrieve the current value of `population`. But there is no current value, because this is the first we're seeing of the variable!

This is only possible with functions, because global variables in Python either exist (with a value) or not (without a value). It's possible for a local variable to be declared, such that Python knows it exists, *but* not (yet) to have a value.

So... what do we do? We want to use the global population, not have a local one.

In [77]:
# option 2: global variable, "population", with the "global" statement

population = 0

class Person:
    def __init__(self, name):
        global population  # this tells Python's compiler not to create a local population, but only to use the global one
        population += 1    # let's add 1 to population
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


In [78]:
# option 3: class attribute on Person

# it's a bad idea to use "global" unless you *really* need to

# classes are objects
# objects have attributes
# we can assign any attribute we want to any object we want, whenever we want
# so... after we define Person, let's just add Person.population = 0!

# we won't have a global "population" variable
# rather, we'll have a "population" attribute on the class that we can access/set


class Person:
    def __init__(self, name):
        Person.population += 1    # let's add 1 to population
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

Person.population = 0     # initialize our population attribute on the class -- Person.population!

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


In [79]:
# option 3: class attribute on Person, defined inside of the class

# option 4:  define the class attribute INSIDE of the class body

class Person:
    population = 0   # this is NOT a variable! It is NOT global! It is, in fact, Person.population, the same class attribute!
    
    def __init__(self, name):
        Person.population += 1    # let's add 1 to population
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


# Why do we want/need class attributes?

1. All methods are class attributes. So if you want methods, they must exist.
2. If you have values that are related to the class, and you don't want them to be floating around outside of the class, then putting them in a class attribute makes sense. This is true for "constants," or generally reused values.
3. If you have a common resource, then it can make sense to put that inside of the class, where everyone can access it.

This is **ABSOLUTELY NOT** a static field! It is not jointly owned or available via the instances and the class! It only exists on the class!

# Exercise: Restricted bowls

1. Modify `Bowl` such that adding a scoop will only work until there are (at most) 3.
2. Any scoops added beyond 3 are ignored / thrown out / melted. Not an error message!
3. Don't hard-code the number 3. Rather, define a class attribute, `MAX_SCOOPS`, that you can reference in the appropriate places.

In [84]:
# usually we want to be forgiving when someone uses == or a similar operator
# with values of different types

class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('cookie dough')
s2 = Scoop('chocolate')
s3 = Scoop('vanilla')
s4 = Scoop('coffee')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')

class Bowl:
    MAX_SCOOPS = 3    # inside of the class definition, but outside of a function -- so we don't need . before the attribute name
    
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        if len(self.scoops) >= Bowl.MAX_SCOOPS:
            return

        print(f'\tAdding scoop of {new_scoop.flavor}...')
        self.scoops.append(new_scoop)

    def flavors(self):
        return f'{self.scoops}'

    def add_scoops(self, *new_scoops):  # new_scoops will be a tuple of Scoop instances
        for one_scoop in new_scoops:
            self.add_scoop(one_scoop)

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
            
        return output

    def __getitem__(self, index):
        return self.scoops[index]

    def __eq__(self, other):
        if isinstance(other, Bowl):    # if these are both instances of Bowl... then check flavors
            return self.flavors() == other.flavors()

        return False

b = Bowl()
b.add_scoop(s1)
b.add_scoop(s2)
b.add_scoop(s3)
b.add_scoop(s4)
b.add_scoop(s5)
b.add_scoop(s6)

print(b)

	Adding scoop of cookie dough...
	Adding scoop of chocolate...
	Adding scoop of vanilla...
Bowl with:
	1: Scoop of cookie dough
	2: Scoop of chocolate
	3: Scoop of vanilla



In [85]:
# it's great that we now have Person.population
# but ... maybe, despite what I said earlier, you think that population is
# shared with the instances. Let's just try it...

class Person:
    population = 0   # this is NOT a variable! It is NOT global! It is, in fact, Person.population, the same class attribute!
    
    def __init__(self, name):
        Person.population += 1    # let's add 1 to population
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')
print(f'After, p1.population = {p1.population}')
print(f'After, p2.population = {p2.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
After, p1.population = 2
After, p2.population = 2
Hello, name1!
Hello, name2!


# Attribute lookup -- ICPO

If you ask an object whether it has an attribute, it'll check and tell you "yes" or "no."

But if the answer is "no," it doesn't stop there. It then checks on its class, to see if the attribute is defined there.

- First check on the **I**nstance
- Then check on the **C**lass